In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.misc import imresize
from scipy.misc import imshow
from scipy.misc import imread
import os
import matplotlib.pyplot as plt
%matplotlib inline

# For fixing the randomness and thereby enabling my team members to reproduce the experiments
np.random.seed(7)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
filename_csv = pd.read_csv('data_chec_resized/dat.csv', sep=',')
filename_csv.head()

In [ ]:
# Uniforming the precision to half sp as to speed up the training (NVIDIA inspired)
temp = []
for img_name in filename_csv.filename:
    img_path = os.path.join('data_chec_resized', 'pool', img_name)
    img = imread(img_path)
    img = imresize(img, (256,192))
    #img = img.astype('float32') 
    temp.append(img)

X = np.stack(temp)

In [ ]:
import keras
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

y = lb.fit_transform(filename_csv['label'])
y = keras.utils.np_utils.to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Initialize the training training data augmentation object
trainAug = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest")
 
# Initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
trainAug.fit(X_train)
train_generator = trainAug.flow(X_train, y_train, batch_size=32)
                                     
valAug.fit(X_test)
valid_generator = valAug.flow(X_test, y_test, batch_size=32)

In [ ]:
NUM_EPOCHS = 64
INIT_LR = 1e-1
BS = 32
def poly_decay(epoch):
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
 
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
    # return the new learning rate
    return alpha

In [ ]:
# Initialize our ResNet model
from resnet import ResNet
model = ResNet.build(192, 256, 3, 8, (3, 4, 6),
                    (16, 32, 64, 128), reg=0.0005)

# Define the optimizer (other variants are also needed to be tried)
from keras.optimizers import SGD, Adam, RMSprop
opt = RMSprop(lr=0.01)

# Compile the model to optimize the categorical_crossentropy as the loss function
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# from keras.callbacks import EarlyStopping, TensorBoard, LearningRateScheduler

# callbacks = [EarlyStopping(monitor='val_loss', patience=5), \
#              TensorBoard(log_dir='./logs'), \
#              LearningRateScheduler(poly_decay)]

In [ ]:
# callbacks = [TensorBoard(log_dir='./logs'), \
#              LearningRateScheduler(poly_decay)]

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="ResNet Model Weights/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
# Fit the model

H = model.fit_generator(
    train_generator,
    steps_per_epoch=2,
    validation_data=valid_generator,
    validation_steps=2,
    epochs=64, callbacks=[checkpoint])

In [ ]:
# plot the training loss and accuracy
N = 64
plt.style.use("ggplot")
plt.figure(figsize=(10,5))
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="uppoer right")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
predictions = model.predict_on_batch(X_test)
print(accuracy_score(y_test, predictions) * 100.)

In [ ]:
X_train.shape

In [ ]:
from keras.applications.vgg16 import VGG16
vgg_conv = VGG16(weights='imagenet',include_top=False,input_shape=(256, 192, 3))

In [ ]:
train_feats=vgg_conv.predict(X_train)
print(train_feats.shape)

In [ ]:
train_features = np.reshape(train_feats, (X_train.shape[0], 8 * 6 * 512))
train_features.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import ClassifierChain

logreg = LogisticRegression()
multi_label_logreg = ClassifierChain(logreg)
multi_label_logreg.fit(train_features, y_train)

In [ ]:
test_feats = vgg_conv.predict(X_test)
print(test_feats.shape)

In [ ]:
test_feats = np.reshape(test_feats, (X_test.shape[0], 8 * 6 * 512))
predictions = multi_label_logreg.predict(test_feats)

In [ ]:
print(accuracy_score(y_test,predictions) * 100.)